# Описание

В общей системе экспериментальной установки нам необходимо подготовить модель сети связи на которую будут наложены сетевые задержки в рамках LWMECPS используя Litmus Chaos. 

Алгоритм HCPP на базе стохастической геометрии позволяет при определенном приближении расчитать сеть связи в двухмерном пространстве.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial.distance import pdist, squareform

class AdvancedHCPPNetwork:
    def __init__(self, area_size=100, num_base_stations=50,
                 min_distance=10, num_users=200):
        """
        Расширенная модель HCPP сети

        Args:
        - area_size: размер территории
        - num_base_stations: число базовых станций
        - min_distance: минимальное расстояние между станциями
        - num_users: число пользователей
        """
        self.area_size = area_size
        self.num_base_stations = num_base_stations
        self.min_distance = min_distance
        self.num_users = num_users

        self.base_stations = None
        self.users = None
        self.connectivity_matrix = None
        self.graph = None

    def generate_base_stations(self):
        """Генерация базовых станций с HCPP"""
        points = []
        attempts = 0
        max_attempts = 1000

        while len(points) < self.num_base_stations and attempts < max_attempts:
            new_point = np.random.uniform(0, self.area_size, 2)

            if not points or all(np.linalg.norm(new_point - p) >= self.min_distance for p in points):
                points.append(new_point)

            attempts += 1

        self.base_stations = np.array(points)
        return self.base_stations

    def generate_users(self, distribution='uniform'):
        """
        Генерация пользователей с различными распределениями

        Args:
        - distribution: 'uniform', 'normal', 'clustered'
        """
        if distribution == 'uniform':
            self.users = np.random.uniform(0, self.area_size, (self.num_users, 2))
        elif distribution == 'normal':
            # Генерация с центрами в случайных точках базовых станций
            center = self.base_stations[np.random.randint(len(self.base_stations))]
            self.users = np.random.normal(center, scale=10, size=(self.num_users, 2))
        elif distribution == 'clustered':
            # Кластеризованное распределение
            clusters = self.base_stations
            self.users = np.concatenate([
                np.random.normal(cluster, scale=5, size=(self.num_users//len(clusters), 2))
                for cluster in clusters
            ])

        return self.users

    def calculate_voronoi_coverage(self):
        """Расчет зон покрытия методом диаграмм Вороного"""
        vor = Voronoi(self.base_stations)

        plt.figure(figsize=(12, 10))
        voronoi_plot_2d(vor, show_vertices=False, line_colors='orange', line_width=2, line_alpha=0.6, point_size=10)
        plt.scatter(self.base_stations[:, 0], self.base_stations[:, 1], c='red', marker='x')
        plt.scatter(self.users[:, 0], self.users[:, 1], c='blue', alpha=0.5, s=10)
        plt.title('Покрытие базовых станций (Диаграмма Вороного)')
        plt.xlabel('X координата')
        plt.ylabel('Y координата')
        plt.show()

    def build_network_graph(self, max_connection_distance=20):
        """
        Построение графа связности базовых станций

        Args:
        - max_connection_distance: максимальное расстояние для соединения
        """
        # Матрица физических расстояний
        distances = squareform(pdist(self.base_stations))

        # Создание графа
        G = nx.Graph()
        G.add_nodes_from(range(len(self.base_stations)))

        # Добавление ребер
        for i in range(len(self.base_stations)):
            for j in range(i+1, len(self.base_stations)):
                if distances[i, j] <= max_connection_distance:
                    G.add_edge(i, j, weight=distances[i, j])

        self.graph = G
        self.connectivity_matrix = nx.adjacency_matrix(G).toarray()

        return self.graph, self.connectivity_matrix

    def visualize_network_graph(self):
        """Визуализация графа связности"""
        plt.figure(figsize=(12, 10))
        pos = {i: self.base_stations[i] for i in range(len(self.base_stations))}
        nx.draw(self.graph, pos, with_labels=True,
                node_color='red', node_size=100,
                font_size=8, font_color='blue')
        plt.title('Граф связности базовых станций')
        plt.show()

# Пример использования
np.random.seed(42)
network = AdvancedHCPPNetwork(
    area_size=100,
    num_base_stations=30,
    min_distance=10,
    num_users=500
)

# Генерация базовых станций и пользователей
base_stations = network.generate_base_stations()
users = network.generate_users(distribution='clustered')

# Визуализация покрытия
network.calculate_voronoi_coverage()

# # Построение графа связности
# graph, connectivity_matrix = network.build_network_graph(max_connection_distance=25)

# # Визуализация графа
# network.visualize_network_graph()

# # Вывод матрицы связности
# print("Матрица связности базовых станций:")
# print(connectivity_matrix)